# All modules fast check 

In [6]:
import main_function as mf


## Set batch name and get data form files
1. 'old' last batch of cells
2. 'Dec' YiHui's new cells from Dec


In [7]:
#mf.batch_name = 'Old'
mf.batch_name = 'Dec'

data = mf.read_all_test()

## Draw overall PCE chart

In [8]:
mf.draw_cell_PCE_overview(data)

## Draw overall PCE ratio chart

In [9]:
mf.draw_cell_PCE_overview_ratio(data)

## Draw all cell performance change ratio

In [10]:
mf.draw_cell_performance(data)